# Get Daily PE Ratios for Tesla and Plot Them

This notebook fetches daily price-to-earnings (PE) ratios for Tesla (TSLA) from the Rice Data Portal.

In [ ]:
from rice_data_client import RiceDataClient
import pandas as pd

# Initialize client with your access token
ACCESS_TOKEN = "your_access_token_here"  # Replace with your token from data-portal.rice-business.org
client = RiceDataClient(access_token=ACCESS_TOKEN)

## Query Daily PE Ratios

Daily PE ratios are in the DAILY table. We'll get the last 2 years of data.

**Note**: The `pe` column contains the price-to-earnings ratio calculated daily.

In [ ]:
# Query daily PE ratios for Tesla
sql = """
SELECT 
    ticker,
    date::DATE as date,
    pe,
    marketcap,
    pb,
    ps
FROM daily
WHERE ticker = 'TSLA'
  AND date::DATE >= CURRENT_DATE - INTERVAL '2 years'
  AND pe IS NOT NULL
ORDER BY date
"""

df = client.query(sql)

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

print(f"Retrieved {len(df)} daily observations")
print(f"Date range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")

In [ ]:
# Preview the data
df.head(10)

In [ ]:
# Recent data
df.tail(10)

## Summary Statistics

In [ ]:
print("\nPE Ratio Summary Statistics for Tesla:")
print(f"Mean PE: {df['pe'].mean():.2f}")
print(f"Median PE: {df['pe'].median():.2f}")
print(f"Min PE: {df['pe'].min():.2f}")
print(f"Max PE: {df['pe'].max():.2f}")
print(f"Std Dev: {df['pe'].std():.2f}")
print(f"\nCurrent PE (most recent): {df['pe'].iloc[-1]:.2f}")

## Save Data as Parquet File

In [ ]:
# What filename would you like to use to save this data?
filename = "tsla_daily_pe_ratios.parquet"

df.to_parquet(filename)
print(f"Data saved to {filename} ({len(df)} rows)")

## Plot Daily PE Ratios

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create figure with subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: PE Ratio over time
ax1.plot(df['date'], df['pe'], linewidth=1.5, color='#E31937', alpha=0.8)
ax1.axhline(y=df['pe'].mean(), color='blue', linestyle='--', linewidth=2, 
            label=f'Mean PE: {df["pe"].mean():.2f}', alpha=0.7)
ax1.axhline(y=df['pe'].median(), color='green', linestyle='--', linewidth=2, 
            label=f'Median PE: {df["pe"].median():.2f}', alpha=0.7)
ax1.set_title('Tesla (TSLA) Daily PE Ratio', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('PE Ratio', fontsize=12)
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Distribution histogram
ax2.hist(df['pe'], bins=50, color='#E31937', alpha=0.7, edgecolor='black')
ax2.axvline(x=df['pe'].mean(), color='blue', linestyle='--', linewidth=2, 
            label=f'Mean: {df["pe"].mean():.2f}')
ax2.axvline(x=df['pe'].median(), color='green', linestyle='--', linewidth=2, 
            label=f'Median: {df["pe"].median():.2f}')
ax2.set_title('Distribution of Tesla PE Ratios', fontsize=14, fontweight='bold')
ax2.set_xlabel('PE Ratio', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Rolling Statistics

In [ ]:
# Calculate 30-day and 90-day rolling averages
df['pe_ma30'] = df['pe'].rolling(window=30).mean()
df['pe_ma90'] = df['pe'].rolling(window=90).mean()

# Plot with moving averages
plt.figure(figsize=(14, 7))
plt.plot(df['date'], df['pe'], linewidth=1, color='#E31937', alpha=0.4, label='Daily PE')
plt.plot(df['date'], df['pe_ma30'], linewidth=2, color='blue', label='30-Day MA')
plt.plot(df['date'], df['pe_ma90'], linewidth=2, color='green', label='90-Day MA')

plt.title('Tesla PE Ratio with Moving Averages', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('PE Ratio', fontsize=12)
plt.legend(loc='best', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Compare Multiple Valuation Metrics

In [ ]:
# Normalize metrics to show on same scale (using z-scores)
from scipy import stats

df_metrics = df[['date', 'pe', 'pb', 'ps']].dropna()

# Calculate z-scores for comparison
df_metrics['pe_z'] = stats.zscore(df_metrics['pe'])
df_metrics['pb_z'] = stats.zscore(df_metrics['pb'])
df_metrics['ps_z'] = stats.zscore(df_metrics['ps'])

# Plot normalized metrics
plt.figure(figsize=(14, 7))
plt.plot(df_metrics['date'], df_metrics['pe_z'], linewidth=1.5, label='PE Ratio', alpha=0.8)
plt.plot(df_metrics['date'], df_metrics['pb_z'], linewidth=1.5, label='PB Ratio', alpha=0.8)
plt.plot(df_metrics['date'], df_metrics['ps_z'], linewidth=1.5, label='PS Ratio', alpha=0.8)
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.title('Tesla Valuation Metrics (Normalized)', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Z-Score', fontsize=12)
plt.legend(loc='best', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nNote: Z-scores show how many standard deviations each metric is from its mean.")
print("This allows comparison of metrics on different scales.")